In [13]:
import torch
import datasets
from torch.utils.data.dataloader import DataLoader,Sampler
import torchvision
from torch import nn


In [31]:
device = 'cuda'

data = datasets.load_dataset('fashion_mnist')
train_data = data['train'].select(range(2000))
print(train_data)
comp = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    #torchvision.transforms.(),
    

])
pos_comp = torchvision.transforms.Compose([
    #torchvision.transforms.RandomVerticalFlip(),
    #torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandAugment(),
    torchvision.transforms.Lambda(lambda x: 2* x/255.0 -1  )
])

#train_data = train_data.map(lambda x :{'image': comp(x['image']) ,'label' :x['label']} ,batch_size=32)
#positive_data = train_data.map(lambda x :{'image': comp(x['image']) ,'label':1} )
#sampler = Sampler(train_data)
batch_size = 100
def custom_collate_fn(batch):
    images = []
    for x in batch:
        img:torch.Tensor = comp(x['image']) 
        img = img.to(torch.uint8)
        images.append(pos_comp(img))
        aug = pos_comp(img)
        images.append(aug)
        
    #images.append(aug)
    
    #rint = torch.randint(0,len(batch),(1,)).numpy()[0]
    #images.append(pos_comp(images[rint]))
    #labels = torch.zeros(size=(len(batch)+1,))
    #labels[rint]=1
    #labels[-1] =1 
    # shaffel images and corresponding label ??
    
    # Convert the list of images to a tensor
    images = torch.stack(images)

    # Shuffle the images and their corresponding labels
    #perm = torch.randperm(images.size(0))
    #images = images[perm]
    #labels = labels[perm]
    
    
    return images.to(device)#, labels


train_loader = DataLoader(train_data, batch_size//2 , collate_fn=custom_collate_fn) #,sampler=sampler)



Dataset({
    features: ['image', 'label'],
    num_rows: 2000
})


In [91]:
for d in train_loader:
    print( d.shape) 
    break

torch.Size([32, 1, 28, 28])


In [27]:
def nt_xent_loss(x, temperature):
  #assert len(x.size()) == 2

  # Cosine similarity
  xcs = torch.cosine_similarity(x[None,:,:], x[:,None,:], dim=-1)
  xcs[torch.eye(x.size(0)).bool()] = float("-inf")

  # Ground truth labels
  target = torch.arange(x.shape[0]).to(device)
  target[0::2] += 1
  target[1::2] -= 1

  # Standard cross-entropy loss
  return torch.nn.functional.cross_entropy(torch.exp( xcs / temperature), target, reduction="mean")

In [32]:
def kl(P):
    #n = torch.distributions.Normal(torch.zeros(out.shape), torch.ones(out.shape))
    #n = torch.Tensor(n)
    #Q = torch.normal(0,1,size=P.shape).to(device)
    #return torch.distributions.kl_divergence(out,n)
    
    #return (P * (P / Q).log()).sum()
    pass

import torchsummary
model = torch.nn.Sequential(
 nn.Conv2d(1, 64, (3, 3), 1, 1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(3),

    nn.Conv2d(64, 256, (3, 3), 1, 1),
    nn.BatchNorm2d(256),
    nn.ReLU(),
    nn.MaxPool2d(3),

    nn.Conv2d(256, 512, (3, 3), 1),
    nn.BatchNorm2d(512),
    nn.ReLU(),
    nn.Flatten(),
    nn.Dropout(0.5),
    
    #torch.nn.Linear(512,128),
    #torch.nn.Tanh()
    
)
model = model.to(device)
torchsummary.summary(model,(1,28,28),device=device)
cross_entropy = torch.nn.CrossEntropyLoss()

optim = torch.optim.RAdam(model.parameters(),lr=1e-4)
def step(x):
    optim.zero_grad()
    out = model.forward(x)
    #print(out.shape)
    t = 0.07
    #z_img = out[0::2]
    #z_aug = out[1::2]
    

    loss = nt_xent_loss(out,t) #+ kl(out) #0.01* torch.sum(out)**2
    


    loss.backward()  
    
    optim.step()
    return loss
    pass
for epochs in range(100):
    for image in train_loader:
        loss = step(image)
        
    if epochs%10==0:
        print(epochs,' :', loss.detach().cpu().numpy())
        pass
    
    


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 28, 28]             640
       BatchNorm2d-2           [-1, 64, 28, 28]             128
              ReLU-3           [-1, 64, 28, 28]               0
         MaxPool2d-4             [-1, 64, 9, 9]               0
            Conv2d-5            [-1, 256, 9, 9]         147,712
       BatchNorm2d-6            [-1, 256, 9, 9]             512
              ReLU-7            [-1, 256, 9, 9]               0
         MaxPool2d-8            [-1, 256, 3, 3]               0
            Conv2d-9            [-1, 512, 1, 1]       1,180,160
      BatchNorm2d-10            [-1, 512, 1, 1]           1,024
             ReLU-11            [-1, 512, 1, 1]               0
          Flatten-12                  [-1, 512]               0
          Dropout-13                  [-1, 512]               0
Total params: 1,330,176
Trainable param

In [11]:
test_data = data['test'].select(range(1000))
test_comp = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Lambda(lambda x: 2*x/255.0-1)
])
def test_collate_fn(batch):
    images = []
    labels = []
    for b in batch:
        images.append(test_comp(b['image']))
        labels.append(b['label'])
        
    images = torch.stack(images).to(device)   
    labels = torch.Tensor(labels).type(torch.LongTensor).to(device)
    return images,labels
    pass 
test_loader = DataLoader(test_data,batch_size,False,collate_fn=test_collate_fn)

next(iter(test_loader))

(tensor([[[[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
           [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
           [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
           ...,
           [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
           [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
           [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000]]],
 
 
         [[[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
           [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
           [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
           ...,
           [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
           [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
           [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000]]],
 
 
         [[[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
       

In [33]:
for p in model.parameters():
    p.requires_grad = False
    
new_model = torch.nn.Sequential(
    model,
    torch.nn.Linear(512,10),
    #torch.nn.Softmax(),
)

new_model= new_model.to(device)

new_optim = torch.optim.RAdam(new_model.parameters(),1e-3)
lossfnc= torch.nn.CrossEntropyLoss()
def new_step(x,y):
    new_optim.zero_grad()
    out = new_model.forward(x)
    loss = lossfnc.forward(out,y)
    loss.backward()
    new_optim.step()    
    return loss
    pass


for epochs in range(20):
    for x,y in test_loader:
        loss = new_step(x,y)
        pass
    print(loss)
    


tensor(2.3500, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2500, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2367, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.0467, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.9907, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.9889, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.9086, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.8062, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.7559, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6798, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6466, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.5727, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.5898, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.5175, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.4898, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.4091, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.3906, device='cuda:0', grad_fn=

tensor(2.3311, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3448, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.1906, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.1696, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.0499, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.0258, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.8667, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.7390, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6636, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6122, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.5382, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.5184, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.3928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.3227, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.3132, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.2306, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.1824, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.1322, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.1127, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.0835, device='cuda:0', grad_fn=<NllLossBackward0>)

In [17]:
m= new_model.eval() 

for x,y in test_loader:
   out = m.forward(x) 
   r = torch.argmax(out,dim=1)
   print(r.shape)
   print((r==y).int().sum()/out.shape[0])
   
   break

torch.Size([100])
tensor(0.8200, device='cuda:0')


In [ ]:
"""    for i in range(0,for_range):
        n = torch.exp(torch.cosine_similarity(z_img[i:i+1], z_aug[i:i+1] )/t)
        
        d= torch.Tensor([0.0]).to(device)
        
        for j in range(0,for_range):
            
            if i!=j:
                d += torch.exp(torch.cosine_similarity(z_img[i:i+1],z_img[j:j+1]))
                pass
        
        loss += torch.log(n/d)
        
        
        
    loss /=- for_range  """